In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie
include("multiharmonic_balance.jl");

# Harmonic Balance using multiple harmonics

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0;
omega = 3;
gamma3 = 0;
g0 = 9.80665; # m / s^2
height = 5; # m
harmonics = 1; # number of harmonics

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
yleft::Float64 = 0.0;
yright::Float64 = 1.0;
Nt = 5
Nx = 3
Ny = 3
order = 2;
stepx = (xright-xleft)/Nx;
stepy = (yright-yleft)/Ny;

In [ ]:
# Define symbolics
Model.@parameters x, y, t;

Dx = Model.Differential(x);
Dy = Model.Differential(y);
Dt = Model.Differential(t);

In [ ]:
u0 = ones((Nx + 1) * (Ny + 1) * harmonics * 2);

In [ ]:
vars, var_exprs, u = create_ansatz((x,y), t, omega, harmonics);
#bcs = create_bcs(vars, ((xleft, xright),), (x,), 0.0)

F = sin(omega*t)# 50 * exp(-40*(x^2))*sin(omega*t)

pde = Dt(Dt(u)) - 9*Dx(Dx(u)) - 9*Dy(Dy(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;

Model.@variables x, y, t;
expanded = expand_trig(pde, t, omega);

eqs = make_residual(expanded, harmonics, omega, t);


In [ ]:
u

In [ ]:
eqs

In [ ]:
using MacroTools: @capture, postwalk, prewalk

In [ ]:
function transform_sym(Nx::Int64, Ny::Int64)
    function aux(ex)
        prewalk(ex) do tmp
            if @capture(tmp, Differential(x)(Differential(x)(s_(x, y))))
                return :(($s[i+1]-2* $s[i]+$s[i-1])/dx^2)
                
            elseif @capture(tmp, Differential(x)(s_(x, y)))
                return :(($s[i+1] - $s[i-1]) / (2*dx))
                
            elseif @capture(tmp, Differential(y)(Differential(y)(s_(x, y))))
                return :(($s[i+Nx+1]-2* $s[i]+$s[i-Nx-1])/dx^2)
                
            elseif @capture(tmp, Differential(x)(s_(x, y)))
                return :(($s[i+Nx+1] - $s[i-Nx-1]) / (2*dx))
                
            elseif @capture(tmp, s_(x, y))
                return :($s[i])
            end
            tmp
        end
    end
    aux
end

In [ ]:
eqs

In [ ]:
sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqs)

In [ ]:
function create_residual_function_1D(Nx::Int64, Ny::Int64, harmonics::Int64, eqs::Vector{Expr})
    Us = [Symbol(Char('A' + i - 1)) for i in 1:2*harmonics]
    Fs = [Symbol(string("F_", Char('A' + i - 1))) for i in 1:2*harmonics]
    N = (Nx + 1) * (Ny + 1) - 1
    quote
        function residual!(F, U, p)
            dx = p

            $([:($v = U[($i-1)*$N+$i:$i*$N+$i]) for (i, v) in enumerate(Us)]...)
            $([:($v = F[($i-1)*$N+$i:$i*$N+$i]) for (i, v) in enumerate(Fs)]...)

            for y in 1:$Ny+1
                for x in 1:$Nx+1
                    i = x + y * ($Nx + 1)
                    if x == 1 || y == 1 || x == $Nx+1 || y == $Ny+1
                        $([:(($f)[i] = ($u)[i]) for (f, u) in zip(Fs, Us)]...)
                    else
                        $([:(($v)[i] = $(eqs[j])) for (j, v) in enumerate(Fs)]...)
                    end
                end
            end

            return F
        end
    end
end

In [ ]:
resid = create_residual_function_1D(Nx, Ny, harmonics, sym_eqs)

In [ ]:
prob = NonlinearSolve.NonlinearProblem(eval(resid), u0, [stepx, stepy])

sol = NonlinearSolve.solve(prob, NewtonRaphson(), reltol=1e-5, abstol=1e-5)
    
    # Extract solution coefficients
solution_coeffs = [sol[vars_symb[i]] for i in 1:(2*harmonics)
                    

u0 = vcat(solution_coeffs...);